In [15]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import GoogleNews
from datetime import date, datetime

In [16]:
Stock = input("Enter the name of the Stock you want to analyze: ")
stock_ticker = input("Enter the ticker representative of the stock: ")

start_date = input("Enter the start date in format(yyyy-mm-dd): ")
end_date = input("Enter the end date in format(yyyy-mm-dd): ")
# Stock = "Apple"
# stock_ticker = "AAPL"
# start_date = "2020-01-01"
# end_date = "2025-01-01"



In [17]:
df = yf.download(
    tickers = stock_ticker,
    start = start_date,
    end = end_date,
    interval = "1d", # for dailyy data
    auto_adjust = False,
    progress = False
)

In [18]:
def data_collection(df):
    print(df.head())
    print(df.tail())
    print(df.columns)
    df.columns = df.columns.get_level_values(0)
    print(df.columns)
    df["H_L"] = df["High"] - df["Low"]

    df["O_C"] = df["Open"] - df["Close"]

    for window in (7,14,21):
        df[f"MA_{window}"] = df["Close"].rolling(window).mean()

    df["STDDEV_7"] = df["Close"].rolling(7).std()

    df = df.dropna()    

    n = len(df)
    split_index = int(n * 0.8)

    train_df = df.iloc[:split_index]
    val_df   = df.iloc[split_index:]

    print("Train range:", train_df.index.min(), "to", train_df.index.max())
    print("Val range:",   val_df.index.min(),   "to", val_df.index.max())
    features = ["H_L", "O_C", "MA_7", "MA_14", "MA_21", "STDDEV_7", "Volume"]
    X_train, y_train = train_df[features], train_df["Close"]
    X_val,   y_val   = val_df[features],   val_df["Close"]

    return X_train, y_train, X_val, y_val, train_df , val_df


In [19]:
# print(df.head())
# print(df.tail())
# print(df.columns)

In [20]:

# df.columns = df.columns.get_level_values(0)
# print(df.columns)


In [21]:
# df["H_L"] = df["High"] - df["Low"]

# df["O_C"] = df["Open"] - df["Close"]

# for window in (7,14,21):
#     df[f"MA_{window}"] = df["Close"].rolling(window).mean()

# df["STDDEV_7"] = df["Close"].rolling(7).std()

# df = df.dropna()

In [22]:
# print(df.index)
# print(df.head())


In [23]:
# n = len(df)
# split_index = int(n * 0.8)

# train_df = df.iloc[:split_index]
# val_df   = df.iloc[split_index:]

# print("Train range:", train_df.index.min(), "to", train_df.index.max())
# print("Val range:",   val_df.index.min(),   "to", val_df.index.max())


In [24]:
X_train, y_train, X_val, y_val, train_df , val_df = data_collection(df)

# print(len(X_train))
# print(len(X_val))
# print(y_train[0])

Price      Adj Close    Close     High      Low     Open     Volume
Ticker          NVDA     NVDA     NVDA     NVDA     NVDA       NVDA
Date                                                               
2020-01-02  5.971745  5.99775  5.99775  5.91800  5.96875  237536000
2020-01-03  5.876163  5.90175  5.94575  5.85250  5.87750  205384000
2020-01-06  5.900805  5.92650  5.93175  5.78175  5.80800  262636000
2020-01-07  5.972245  5.99825  6.04425  5.90975  5.95500  314856000
2020-01-08  5.983445  6.00950  6.05100  5.95375  5.99400  277108000
Price        Adj Close       Close        High         Low        Open  \
Ticker            NVDA        NVDA        NVDA        NVDA        NVDA   
Date                                                                     
2024-12-24  140.197372  140.220001  141.899994  138.649994  140.000000   
2024-12-26  139.907410  139.929993  140.850006  137.729996  139.699997   
2024-12-27  136.987885  137.009995  139.020004  134.710007  138.550003   
2024-12-30  

## ANN


In [25]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error

import matplotlib.pyplot as plt


In [26]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)

In [27]:
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]),  # hidden layer
    Dense(1)  # output layer for price prediction
])

model.compile(optimizer='adam', loss='mse')


/home/billy_56/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=500,
    batch_size=32,
    # callbacks=[early_stop],
    verbose=2
)

Epoch 1/500
31/31 - 1s - 22ms/step - loss: 585.2948 - val_loss: 11426.3301
Epoch 2/500
31/31 - 0s - 4ms/step - loss: 542.7349 - val_loss: 9968.3633
Epoch 3/500
31/31 - 0s - 4ms/step - loss: 496.9443 - val_loss: 8378.9922
Epoch 4/500
31/31 - 0s - 5ms/step - loss: 445.4932 - val_loss: 6526.1699
Epoch 5/500
31/31 - 0s - 5ms/step - loss: 387.7156 - val_loss: 4655.1372
Epoch 6/500
31/31 - 0s - 5ms/step - loss: 326.4859 - val_loss: 2893.0781
Epoch 7/500
31/31 - 0s - 7ms/step - loss: 265.6304 - val_loss: 1437.1875
Epoch 8/500
31/31 - 0s - 6ms/step - loss: 208.1596 - val_loss: 543.6304
Epoch 9/500
31/31 - 0s - 6ms/step - loss: 159.5280 - val_loss: 334.6058
Epoch 10/500
31/31 - 0s - 6ms/step - loss: 121.5043 - val_loss: 805.2591
Epoch 11/500
31/31 - 0s - 4ms/step - loss: 94.6277 - val_loss: 1781.7563
Epoch 12/500
31/31 - 0s - 4ms/step - loss: 76.5177 - val_loss: 2826.5923
Epoch 13/500
31/31 - 0s - 5ms/step - loss: 65.5354 - val_loss: 3768.7900
Epoch 14/500
31/31 - 0s - 4ms/step - loss: 57.9085 

In [29]:

y_pred_ann = model.predict(X_val_scaled).flatten()

rmse = np.sqrt(mean_squared_error(y_val, y_pred_ann))
mae  = mean_absolute_error(y_val, y_pred_ann)

print(f"ANN RMSE: {rmse:.3f}")
print(f"ANN MAE : {mae:.3f}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
ANN RMSE: 7.112
ANN MAE : 4.503


In [30]:


fig = go.Figure()


fig.add_trace(go.Scatter(
    x=val_df.index,
    y=y_val.values,
    mode='markers+lines',
    name='Actual',
    marker=dict(symbol='square', size=8, color='blue'),
    line=dict(color='blue')
))


fig.add_trace(go.Scatter(
    x=val_df.index,
    y=y_pred_ann,
    mode='markers+lines',
    name='Predicted (ANN)',
    marker=dict(symbol='triangle-up', size=8, color='orange'),
    line=dict(color='orange', dash='dot')
))

fig.update_layout(
    title=f"Actual vs Predicted Close Price for {Stock} (ANN)",
    xaxis_title="Date",
    yaxis_title="Price",
    hovermode="x unified",
    template="plotly_white"
)

fig.show()


In [31]:
today = date.today()
two_years = today.replace(year=today.year - 2)


df = yf.download(
    tickers = stock_ticker,
    start = two_years,
    end = today,
    interval = "1d", # for dailyy data
    auto_adjust = False,
    progress = False
)

X_train, y_train, X_val, y_val, train_df , val_df = data_collection(df)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)

model = Sequential([
    Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]),  # hidden layer
    Dense(1)  # output layer for price prediction
])

model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=500,
    batch_size=32,
    # callbacks=[early_stop],
    verbose=2
)


y_pred_ann = model.predict(X_val_scaled).flatten()

rmse = np.sqrt(mean_squared_error(y_val, y_pred_ann))
mae  = mean_absolute_error(y_val, y_pred_ann)

print(f"ANN RMSE: {rmse:.3f}")
print(f"ANN MAE : {mae:.3f}")



# get only the last 7 entries
last7_df    = val_df.tail(7)
last7_actual = y_val.tail(7)
last7_pred   = y_pred_ann[-7:]   # assuming y_pred_ann is aligned in order

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=last7_df.index,
    y=last7_actual.values,
    mode='markers+lines',
    name='Actual',
    marker=dict(symbol='square', size=8, color='blue'),
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=last7_df.index,
    y=last7_pred,
    mode='markers+lines',
    name='Predicted (ANN)',
    marker=dict(symbol='triangle-up', size=8, color='orange'),
    line=dict(color='orange', dash='dot')
))

fig.update_layout(
    title=f"Actual vs Predicted Close Price for {Stock} (ANN) — Last 7 Days",
    xaxis_title="Date",
    yaxis_title="Price",
    hovermode="x unified",
    template="plotly_white"
)

fig.show()

Price       Adj Close      Close       High        Low       Open     Volume
Ticker           NVDA       NVDA       NVDA       NVDA       NVDA       NVDA
Date                                                                        
2023-07-11  42.378918  42.404999  42.757999  42.067001  42.480999  298244000
2023-07-12  43.875000  43.902000  43.935001  42.777000  43.033001  481277000
2023-07-13  45.948727  45.977001  46.154999  44.492001  44.518002  478204000
2023-07-14  45.441040  45.469002  48.088001  45.060001  46.583000  772075000
2023-07-17  46.432423  46.460999  46.495998  45.262001  46.289001  510488000
Price        Adj Close       Close        High         Low        Open  \
Ticker            NVDA        NVDA        NVDA        NVDA        NVDA   
Date                                                                     
2025-07-03  159.339996  159.339996  160.979996  157.770004  158.369995   
2025-07-07  158.240005  158.240005  159.309998  157.339996  158.199997   
2025-07-08  16

/home/billy_56/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



13/13 - 1s - 50ms/step - loss: 10104.0283 - val_loss: 16178.2988
Epoch 2/500
13/13 - 0s - 9ms/step - loss: 10030.5088 - val_loss: 16069.1006
Epoch 3/500
13/13 - 0s - 8ms/step - loss: 9955.6475 - val_loss: 15954.0977
Epoch 4/500
13/13 - 0s - 8ms/step - loss: 9874.8398 - val_loss: 15831.1338
Epoch 5/500
13/13 - 0s - 8ms/step - loss: 9788.4648 - val_loss: 15690.6973
Epoch 6/500
13/13 - 0s - 8ms/step - loss: 9692.1211 - val_loss: 15529.0859
Epoch 7/500
13/13 - 0s - 8ms/step - loss: 9579.0723 - val_loss: 15340.7793
Epoch 8/500
13/13 - 0s - 8ms/step - loss: 9459.8184 - val_loss: 15148.2744
Epoch 9/500
13/13 - 0s - 8ms/step - loss: 9333.7646 - val_loss: 14935.3477
Epoch 10/500
13/13 - 0s - 8ms/step - loss: 9191.9736 - val_loss: 14682.3037
Epoch 11/500
13/13 - 0s - 8ms/step - loss: 9032.2520 - val_loss: 14413.8398
Epoch 12/500
13/13 - 0s - 8ms/step - loss: 8869.5850 - val_loss: 14130.1543
Epoch 13/500
13/13 - 0s - 8ms/step - loss: 8696.0137 - val_loss: 13835.7852
Epoch 14/500
13/13 - 0s - 7ms/

In [32]:
val_df = val_df.copy()
val_df["Predicted"] = y_pred_ann  # Your ANN predictions
val_df["Error"] = np.abs(y_val.values - y_pred_ann)


In [33]:
val_df.to_csv("val_with_predictions.csv")


In [34]:
model.save("ann_model.keras")  # no warning!
